In [1]:
# Install ctransformers and dependencies# Install dependencies (Colab-safe)
!pip install -q ctransformers langgraph langchain faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [3]:
from ctransformers import AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal, List

# Load quantized LLM using ctransformers (fast and CPU-friendly)
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    model_file="mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    model_type="mistral",
    context_length=2048,
    gpu_layers=0  # CPU only
)

# Documents: You can replace with your own domain-specific data
docs = [
    "User A enjoys science fiction and prefers short books.",
    "Book: Dune – A classic sci-fi epic.",
    "Book: Project Hail Mary – A fast-paced science fiction novel.",
    "Book: The Hobbit – A short fantasy story with adventure.",
    "Book: Foundation – A legendary sci-fi series by Isaac Asimov.",
    "User B enjoys romance and character-driven narratives.",
    "Book: Pride and Prejudice – A romantic classic.",
    "Book: The Notebook – A deeply emotional romance.",
]

# Vector store (RAG backend)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
faiss_docs = splitter.create_documents(docs)
vectorstore = FAISS.from_documents(faiss_docs, embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LangGraph Agent State
class AgentState(TypedDict):
    user_query: str
    retrieved_docs: List[str]
    final_response: str
    next_action: Literal["retrieve", "reason", "end"]

# Planner Node (decides what to do)
def planner_node(state: AgentState) -> AgentState:
    if not state.get("retrieved_docs"):
        next_action = "retrieve"
    elif state.get("final_response"):  # Already has a response
        next_action = "end"
    else:
        next_action = "reason"
    return {**state, "next_action": next_action}


# Retrieval Tool
def retrieval_node(state: AgentState) -> AgentState:
    docs = retriever.get_relevant_documents(state["user_query"])
    return {
        **state,
        "retrieved_docs": [doc.page_content for doc in docs],
        "next_action": "reason"
    }

# Reasoning Tool (LLM)
def reasoning_node(state: AgentState) -> AgentState:
    context = "\n".join(state["retrieved_docs"])
    prompt = f"""You are a recommendation system.

Only use the information from the context below to answer the user. Do not use any external knowledge.

User Query:
{state['user_query']}

Context:
{context}

Answer with a personalized recommendation using only the above context.
"""
    response = llm(prompt)
    return {
        **state,
        "final_response": response.strip(),
        "next_action": "end"
    }

# LangGraph Agentic Flow
builder = StateGraph(AgentState)
builder.add_node("planner", planner_node)
builder.add_node("retrieve", retrieval_node)
builder.add_node("reason", reasoning_node)

builder.set_entry_point("planner")

builder.add_conditional_edges("planner", lambda s: s["next_action"], {
    "retrieve": "retrieve",
    "reason": "reason",
    "end": END
})

builder.add_edge("retrieve", "planner")
builder.add_edge("reason", "planner")

graph = builder.compile()

# Run the agent
query = "Can you suggest a fast sci-fi book for someone who likes short stories?"
result = graph.invoke({"user_query": query})

print("🤖 Final Answer:")
print(result["final_response"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-3-cf5f0d2d7cc2>:56: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(state["user_query"])


🤖 Final Answer:
Based on your preference for science fiction and short books, I would recommend the Foundation series by Isaac Asimov. This collection of interconnected short stories offers an intriguing exploration into psychohistory, a unique blend of history and mathematics, as it charts the fall and rise of the Galactic Empire. Enjoy your reading!
